# Dynamic Snippet

Trying to solve the following dynamic system:

1. Particle with an initial velocity
2. Affected by its weight (W) and drag (D)

![problem diagram](diagram.png)

Create a `Dynamics` instance.

In [1]:
from app.dynamics import Dynamics

dynamics: Dynamics = Dynamics()

Declare the coordinates that define our problem. These are just keys, they can be anything. But, since they might be reused, it might be better to declare them as constants.

In [2]:
X_KEY: str      = 'X'
Y_KEY: str      = 'Y'
X_DOT_KEY: str  = 'X_DOT'
Y_DOT_KEY: str  = 'Y_DOT'
MASS_KEY: str   = 'MASS'

Create our first `Contributor` to the dynamics sytem: the weight of the particle (gravity).

In [3]:
from app.contributor import Contributor
from app.contributors.gravity import UnidirectionalGravity

gravity: Contributor = UnidirectionalGravity(
    acceleration=-9.81,  # Negative acceleration since +Y is defined as 'UP'
    z_key=Y_KEY,
    z_dot_key=Y_DOT_KEY,
)

Adding it to the dynamcis system will already start defining the state vector. This is done "under the hood" by the `Coordinates` class.

In [4]:
dynamics.add_contributor(gravity)

print(f"Coor  ={dynamics.coordinates._keys}")

Coor  =['Y', 'Y_DOT']


And the second `Contributor`, the drag: 

In [5]:
from app.contributors.drag import ConstantCdAndAreaDrag

drag: Contributor = ConstantCdAndAreaDrag(
    cd=2.2,
    area=10.0,
    x_key=X_KEY,
    y_key=Y_KEY,
    x_dot_key=X_DOT_KEY,
    y_dot_key=Y_DOT_KEY,
    mass_key=MASS_KEY,
)


And again, the state vector would have increased (only with the non-overlapping coordinates):

In [6]:
dynamics.add_contributor(drag)

print(f"Coor  ={dynamics.coordinates._keys}")

Coor  =['Y', 'Y_DOT', 'X', 'X_DOT', 'MASS']


Define our initial state vector:

In [7]:
from app.coordinates import Coordinates

coordinates: Coordinates = dynamics.coordinates
x = [0]*coordinates.size()
x_dot = [0]*coordinates.size()

x[coordinates.get_index(X_KEY)] = 0.0
x[coordinates.get_index(Y_KEY)] = 100.0
x[coordinates.get_index(X_DOT_KEY)] = -10.0
x[coordinates.get_index(Y_DOT_KEY)] = 30.0
x[coordinates.get_index(MASS_KEY)] = 5.0

print(f"Coor  ={coordinates._keys}")
print(f"x     ={x}")
print(f"x_dot ={x_dot}")

Coor  =['Y', 'Y_DOT', 'X', 'X_DOT', 'MASS']
x     =[100.0, 30.0, 0.0, -10.0, 5.0]
x_dot =[0, 0, 0, 0, 0]


The function below mirrors OSTk `Dynamics` signature for comparison:

```
void SatelliteDynamics::DynamicalEquations(const Dynamics::StateVector& x, Dynamics::StateVector& dxdt, const double t)
```

This is essentially what is called during each integration step. Calling it below just as as an example:

In [8]:
dynamics.get_dynamics_equation(x=x, x_dot=x_dot)

print(f"Coor  ={coordinates._keys}")
print(f"x     ={x}")
print(f"x_dot ={x_dot}")

Coor  =['Y', 'Y_DOT', 'X', 'X_DOT', 'MASS']
x     =[100.0, 30.0, 0.0, -10.0, 5.0]
x_dot =[0, 2555.9310834006747, 0, -855.2470278002249, 0]
